In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2
# %matplotlib notebook

# external imports
import numpy as np
import sympy as sp
# from scipy.linalg import solve_discrete_are
import matplotlib.pyplot as plt

# internal imports
from nonlinear_dynamics import simulate
from mld_dynamics import mld, h
from warm_start_hmpc.controller import HybridModelPredictiveController
from warm_start_hmpc.mcais import mcais, solve_dare
from visualizer import vis, animate

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7000/static/


In [2]:
# controller horizon
T = 20

# weight matrices
C = np.eye(mld.nx) * h
D = np.vstack([1.]+[0.]*(mld.nu - 1)).T * h

# # terminal cost
# Bu = mld.B[:,0:1]
# Du = D[:,0:1]
# P = solve_discrete_are(mld.A, Bu, C.T.dot(C), Du.T.dot(Du))
# C_T = np.linalg.cholesky(P).T
# C_T = C
# objective = [C, D, C_T]

# # terminal constraints
# F_T = np.vstack((np.eye(mld.nx), - np.eye(mld.nx)))
# h_T = np.concatenate((x_max, x_max))
# terminal_set = [F_T, h_T]

In [3]:
from pympc.dynamics.discrete_time_systems import LinearSystem
from pympc.geometry.polyhedron import Polyhedron

# LQR cost to go
Bu = mld.B[:,0:1]
Du = D[:,0:1]
P, K = solve_dare(mld.A, Bu, C.dot(C), Du.dot(Du))
C_T = np.linalg.cholesky(P).T
objective = [C, D, C_T]

# # mcais terminal set
# domain = Polyhedron(
#     np.hstack((mld.F, mld.G[:,0:1])),
#     mld.h
# )
# X_T = S.mcais(K, domain, verbose=True)
# terminal_set = [X_T.A, X_T.b]

In [4]:
A_cl = mld.A + Bu.dot(K)
lhs = mld.F + mld.G[:,0:1].dot(K)
rhs = mld.h
terminal_set = mcais(A_cl, lhs, rhs, verbose=True)

Academic license - for non-commercial use only
Time horizon: 42. Convergence index: 0.000000. Number of facets: 216.
Maximal constraint-admissible invariant set found. Removing redundant facets ... minimal facets are 90.


In [5]:
# hybrid controller
controller = HybridModelPredictiveController(mld, T, objective, terminal_set)

# initial push towards the right wall
x0 = np.array([0., 0., 1., 0.])

## Open-loop solution

In [6]:
# solve in open loop
solution, leaves = controller.feedforward(
    x0,
    draw_label='Cart pole with walls'
)

|    Updates    |    Time (s)   |  Solved nodes |  Lower bound  |  Upper bound  |
 --------------- --------------- --------------- --------------- ---------------
 Root node       0.05            1               4.266e-02       inf             
                 3.07            95              6.147e-02       inf             
 New incumbent   6.06            184             6.214e-02       6.214e-02       
 Solution found  6.06            184             6.214e-02       6.214e-02       

Explored 184 nodes in 6.065 seconds: optimal solution found with cost 6.214e-02.


In [7]:
x_gurobi, obj_gurobi = controller.feedforward_gurobi(x0)

Changed value of parameter OutputFlag to 1
   Prev: 0  Min: 0  Max: 1  Default: 1
Optimize a model with 894 rows, 224 columns and 2608 nonzeros
Model has 110 quadratic objective terms
Variable types: 144 continuous, 80 integer (80 binary)
Coefficient statistics:
  Matrix range     [6e-04, 2e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [5e-03, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e-01, 2e+02]
Presolve removed 611 rows and 78 columns
Presolve time: 0.02s
Presolved: 283 rows, 146 columns, 1273 nonzeros
Presolved model has 104 quadratic objective terms
Variable types: 111 continuous, 35 integer (35 binary)

Root relaxation: objective 5.679353e-02, 445 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.05679    0   26          -    0.05679      -     -    0s
     0     0    0.05679    0   26          -    0.0567

In [8]:
vis.jupyter_cell()

In [10]:
animate(solution.variables['x'], h)

## Solve with and without warm start

In [11]:
e0 = np.random.randn(mld.nx) * 0.
uc0 = solution.variables['uc'][0]
ub0 = solution.variables['ub'][0]
warm_start = controller.construct_warm_start(leaves, x0, uc0, ub0, e0)

In [12]:
u0 = np.concatenate((uc0, ub0))
x1 = mld.A.dot(x0) + mld.B.dot(u0) + e0
solution, leaves = controller.feedforward(
    x1,
    warm_start=warm_start,
#     draw_label='Cart pole with walls warm start'
)

Loaded warm start with 93 nodes. Lower bound from warm start is 0.000.
|    Updates    |    Time (s)   |  Solved nodes |  Lower bound  |  Upper bound  |
 --------------- --------------- --------------- --------------- ---------------
 New incumbent   0.83            25              5.391e-02       5.391e-02       
 Solution found  0.83            25              5.391e-02       5.391e-02       

Explored 25 nodes in 0.835 seconds: optimal solution found with cost 5.391e-02.


In [13]:
solution, leaves = controller.feedforward(x1)

|    Updates    |    Time (s)   |  Solved nodes |  Lower bound  |  Upper bound  |
 --------------- --------------- --------------- --------------- ---------------
 Root node       0.03            1               3.931e-02       inf             
                 3.05            99              5.376e-02       inf             
                 6.07            183             5.391e-02       inf             
 New incumbent   6.30            190             5.391e-02       5.391e-02       
 Solution found  6.30            190             5.391e-02       5.391e-02       

Explored 190 nodes in 6.304 seconds: optimal solution found with cost 5.391e-02.


## Solve in closed loop

In [14]:
from visualizer import vis, visualize
vis.jupyter_cell()

In [15]:
# np.random.seed(1)

# set up simulation
T_sim = 30
x_sim = [x0]
u_sim = []
warm_start = None

for t in range(T_sim):
    print('Time step %d.'%t)
    
    # solve miqp
    solution, leaves = controller.feedforward(
        x_sim[-1],
        warm_start=warm_start,
#         printing_period=None
    )
    
    # reorganize solution
    uc0 = solution.variables['uc'][0]
    ub0 = solution.variables['ub'][0]
    u0 = np.concatenate((uc0, ub0))
    x1 = simulate(x_sim[-1], h, uc0[0])[-1]
#     x1 = solution.variables['x'][1]
    e0 = x1 - mld.A.dot(x_sim[-1]) - mld.B.dot(u0)
    print('predicted state:', solution.variables['x'][1])
    print('true state:', x1)
    print('modeling error:', e0)
    visualize(x1)
    
    # generate warm start
    warm_start = controller.construct_warm_start(leaves, x_sim[-1], uc0, ub0, e0)
    
    # retrieve closed-loop trajectory
    x_sim.append(x1)
    u_sim.append(uc0)

Time step 0.
|    Updates    |    Time (s)   |  Solved nodes |  Lower bound  |  Upper bound  |
 --------------- --------------- --------------- --------------- ---------------
 Root node       0.04            1               4.266e-02       inf             
                 3.06            86              5.864e-02       inf             
                 6.06            176             6.214e-02       inf             
 New incumbent   6.32            184             6.214e-02       6.214e-02       
 Solution found  6.32            184             6.214e-02       6.214e-02       

Explored 184 nodes in 6.319 seconds: optimal solution found with cost 6.214e-02.
predicted state: [ 0.05        0.          0.92435119 -0.07564881]
true state: [ 0.04814312 -0.00186037  0.92405416 -0.076243  ]
modeling error: [-0.00185688 -0.00186037 -0.00029703 -0.00059419]
Time step 1.
Loaded warm start with 93 nodes. Lower bound from warm start is 0.000.
|    Updates    |    Time (s)   |  Solved nodes |  Lo

KeyboardInterrupt: 

## Animation

In [ ]:
from visualizer import vis, animate
vis.jupyter_cell()

In [ ]:
animate(x_sim, h)